# <center> Final training of Age estimation model </center>



In [ ]:
!pip install GitPython
from git import Repo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 8.8 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 


In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=81475894bfd31637641de194b8b1312a09fb6bc5e2398abd0321e5535ceeef1c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json'

kaggle.json
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!kaggle datasets download -d bjoernjostein/georgia-12lead-ecg-challenge-database
!kaggle datasets download -d bjoernjostein/st-petersburg-incart-12lead-arrhythmia-database
!kaggle datasets download -d bjoernjostein/china-12lead-ecg-challenge-database
!kaggle datasets download -d bjoernjostein/china-physiological-signal-challenge-in-2018
!kaggle datasets download -d erarayamorenzomuten/chapmanshaoxing-12lead-ecg-database
!kaggle datasets download -d erarayamorenzomuten/ningbo-first-hospital-12lead-ecg-database

!kaggle datasets download -d bjoernjostein/physionet-challenge-2021-snomed-mappings

 98% 553M/564M [00:08<00:00, 68.0MB/s]
100% 564M/564M [00:08<00:00, 67.7MB/s]


In [ ]:
import os, zipfile

dir_name = "./"
extension = ".zip"
 
os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
import shutil
import os
os.mkdir("/content/ecg_age/data")
sources = ['Training_2/',  'Training_StPetersburg/','Training_WFDB/','WFDB_ChapmanShaoxing/', 'WFDB_Ningbo/', "WFDB/"]
dest1 = '/content/ecg_age/data/'


for s in sources:
  files = os.listdir(s)
  for f in files:
      shutil.move(s+f, dest1)

In [ ]:
from ecg_age.src.helpers.helpers import *
from ecg_age.src.models.models import *

In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tqdm
from scipy import signal
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from scipy.io import loadmat
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload
%reload_ext autoreload
sns.set_style("dark")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
######################################################
print("Collecting labels, ECG filenames and metadata...")
gender, age, labels, ecg_len, ecg_filenames = import_key_data("/content/ecg_age/data/")
ecg_filenames = np.asarray(ecg_filenames)
age = np.asarray(age)
gender = np.asarray(gender)
ecg_len = np.asarray(ecg_len)
labels = np.asarray(labels)
print(f"Total number of patients found: {len(age)}")

100%|██████████| 131800/131800 [02:56<00:00, 747.55it/s]

Total number of patients found: 59490


In [ ]:
#####################################################
print("remove all ECGs not equal to 10 seconds")
age, gender,ecg_filenames, labels = only_ten_sec(ecg_len, age, gender, ecg_filenames, labels)

remove all ECGs not equal to 10 seconds


In [ ]:
print("remove all ECGs with missing values in gender or age")
ecg_filenames, gender, age, labels = remove_nan_and_unknown_values(ecg_filenames, gender, age, labels)

In [ ]:
age = clean_up_age_data(age)
gender = clean_up_gender_data(gender)

print(f"Total number of patients left after data cleaning: {len(age)}")

Total number of patients left after data cleaning: 59355


In [ ]:
print("Select ECGs from males or females")
gender_choice = "female"

if gender_choice == "female":
    gender, age, ecg_filenames, labels = male_or_female(gender, age, ecg_filenames, labels g="female")
elif gender_choice == "male":
    gender, age, ecg_filenames, labels = male_or_female(gender, age, ecg_filenames, labels g="male")
else:
    print("Train on both male and female")
    pass

In [ ]:
print(f"Total number of patients left after selection and data cleaning: {len(age)}")

Total number of patients left after data cleaning: 26012


In [ ]:
#TODO: update this acording to the optimal learning rate schedule found in hyperparameter opt.
def scheduler(epoch, lr):
    if epoch == 10:
        return lr * 0.1
    elif epoch == 15:
        return lr * 0.1
    elif epoch == 20:
        return lr * 0.1
    else:
        return lr

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0)

In [ ]:
model_name = "attia" # or "inception"

In [ ]:
samp_freq = 100
time = 10
num_leads = 12
batchsize = 16
epoch = 20
mae_score_total = []

print("Training model...")

#Chose model:
if model_name == "attia":
    model = attia_model(samp_freq, time, num_leads)
elif model_name == "inception":
    model = build_model((samp_freq*time,num_leads), 1)
model.fit(x=shuffle_batch_generator_age(batch_size=batchsize, gen_x=generate_X_age(ecg_filenames,samp_freq=samp_freq,num_leads=num_leads), gen_y=generate_y_age(age),num_leads=num_leads, time=time, samp_freq=samp_freq), epochs=epoch, 
            steps_per_epoch=(len(ecg_filenames)/batchsize), 
            verbose = 1, 
            callbacks=[lr_schedule]
            )

Training model...
Epoch 1/20
1083/1083 [==============================] - 143s 119ms/step - loss: 1342.2295 - mean_absolute_error: 29.7201 - val_loss: 214.4473 - val_mean_absolute_error: 11.5169 - lr: 0.0010
Epoch 2/20
1083/1083 [==============================] - 60s 56ms/step - loss: 238.8643 - mean_absolute_error: 12.0604 - val_loss: 179.4969 - val_mean_absolute_error: 10.3433 - lr: 0.0010
Epoch 3/20
1083/1083 [==============================] - 60s 55ms/step - loss: 220.2294 - mean_absolute_error: 11.5442 - val_loss: 221.8054 - val_mean_absolute_error: 11.8596 - lr: 0.0010
Epoch 4/20
1083/1083 [==============================] - 61s 56ms/step - loss: 209.3514 - mean_absolute_error: 11.2443 - val_loss: 166.7784 - val_mean_absolute_error: 10.0193 - lr: 0.0010
Epoch 5/20
1083/1083 [==============================] - 64s 60ms/step - loss: 200.7987 - mean_absolute_error: 10.9663 - val_loss: 196.4565 - val_mean_absolute_error: 10.7758 - lr: 0.0010
Epoch 6/20
1084/1083 [======================

In [ ]:
model.save_weights("model_weights_" + model_name + gender_choice + ".h5")